In [ ]:
import copy
import numpy as np
import pandas as pd
import pandas_profiling as pdp
from IPython.core.display import display
from datetime import datetime
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 2000)

In [ ]:
import xgboost as xgb
from sklearn.svm import LinearSVC, libsvm, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score

In [ ]:
# 自作の説明変数を作成する関数
def create_explain_variable(df):
    df_add_var = copy.deepcopy(df)
    # キャンペーンの丸め
    df_add_var.loc[20 < df_add_var.campaign, "campaign"] = 21
    # duration の対数変換
    df_add_var.loc[df_add_var.duration==0, "duration"] = 1e-7
    df_add_var.duration = df_add_var.duration.apply(lambda x: np.log(x))
    # 月+日
    dict_month = {"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12,"jan":1,"feb":2,"mar":3}
    df_add_var.month = df_add_var.month.apply(lambda x: dict_month[x]).astype(str)
    df_add_var["md"] = (df_add_var.month + df_add_var.day.apply(lambda x: "%02d" % x)).astype(int)
    df_add_var.month = df_add_var.month.astype(int)
    # 21歳から働き始めたとみなして、平均年間貯蓄額を算出（21歳以下、学生は0とみなす）
    df_add_var["mean_balance"] = df_add_var.balance / (df_add_var.age - 21)
    df_add_var.loc[(df_add_var.age <= 21)|(df_add_var.job=="student"), "mean_balance"] = 0
    # 平均接触回数
    df_add_var["mean_meet_num"] = df_add_var.previous / df_add_var.pdays
    df_add_var.loc[df_add_var.pdays==-1, "mean_meet_num"] = 0
    return df_add_var
    
def add_col_high_rate(df_train, df_test):
    # 可能性の高そうな職業などについている人に点数をつける
    def calc_high_rate(df, col, threshold=0.12):
        y_cnt  = df.groupby(col, as_index=False).y.count()
        y_sum  = df.groupby(col, as_index=False).y.sum()
        y_rate = pd.merge(y_sum, y_cnt, on=col, suffixes=("_sum","_cnt"))
        y_rate["rate"] = y_rate.y_sum / y_rate.y_cnt
        return list(y_rate.query("@threshold < rate")[col])
    dict_high_rate = {}
    dict_high_rate["job"]       = calc_high_rate(df_train, "job")
    dict_high_rate["marital"]   = calc_high_rate(df_train, "marital")
    dict_high_rate["education"] = calc_high_rate(df_train, "education")
    dict_high_rate["poutcome"]  = ["success"]
    df_train["high_rate"] = 0
    df_test["high_rate"]  = 0
    for col, vals in dict_high_rate.items():
        df_train.loc[df_train[col].isin(vals), "high_rate"] = df_train.high_rate + 2
        df_test.loc[df_test[col].isin(vals),   "high_rate"] = df_test.high_rate + 2
    return df_train, df_test

In [ ]:
# カテゴリデータのダミー変数化
def dummies(df, cols):
    df_droped  = copy.deepcopy(df.drop(cols, axis=1)).reset_index(drop=True)
    df_dummies = df.reset_index(drop=True)
    df_dummies = pd.get_dummies(df_dummies[cols], drop_first=True)
    return pd.merge(df_droped, df_dummies, left_index=True, right_index=True)

# 数値データの標準化
def standardization(df, cols, df_test=None):
    mean   = df[cols].mean()
    std    = df[cols].std()
    df_std = copy.deepcopy(df)
    df_std[cols] = df_std[cols].apply(lambda x: (x - mean[x.name]) / std[x.name])
    df_test_std  = copy.deepcopy(df_test)
    if df_test is not None:
        df_test_std[cols] = df_test_std[cols].apply(lambda x: (x - mean[x.name]) / std[x.name])
    return df_std, df_test_std

In [ ]:
path = "../../../../study/bank/motodata/"
df_train = pd.read_csv(path + "train.csv")
df_test  = pd.read_csv(path + "test.csv")

In [ ]:
# 外れ値データを削除（全員のモデルを足すときは実行していない）
dict_del = {"previous":[275],
            "balance" :[-6847,102127]}
for col, vals in dict_del.items():
    for val in vals:
        del_idx = df_train[df_train[col]==val].index
        df_train.drop(del_idx, inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [ ]:
# 自作変数
df_train_add_var = create_explain_variable(df_train)
df_test_add_var  = create_explain_variable(df_test)
df_train_add_var, df_test_add_var = add_col_high_rate(df_train_add_var, df_test_add_var) 

In [ ]:
# 不要な列を削除
drop_cols = ["id","month"]
df_train_add_var.drop(drop_cols, axis=1, inplace=True)
df_test_add_var.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# 標準化
std_cols = df_train_add_var.select_dtypes(include=["int","float"]).columns
std_cols = std_cols.drop("y")
df_train_add_var, df_test_add_var = standardization(df_train_add_var, std_cols, df_test_add_var)
# ダミー変数化
dummies_cols = df_train_add_var.select_dtypes(include="object").columns
df_dummies   = pd.concat([df_train_add_var, df_test_add_var])
df_dummies   = dummies(df_dummies, dummies_cols)
df_train_add_var, df_test_add_var = df_dummies[df_dummies.y.notnull()], df_dummies[df_dummies.y.isnull()]
df_train_add_var.y = df_train_add_var.y.astype(int)
df_test_add_var.drop("y", axis=1, inplace=True)

In [ ]:
display(df_train_add_var.head())
display(df_test_add_var.head())

In [ ]:
pdp.ProfileReport(df_train_add_var)

In [ ]:
# スタッキングする関数
def calc_proba(proba):
    return 1 - proba[:,0]
def stacking(df_train, df_test, clf, name, seed=15, cv=4):
    statime = datetime.now()
    df_auc  = df_train_pred = pd.DataFrame()
    x, y    = df_train.drop("y", axis=1), df_train.y
    # train
    k   = 1
    skf = StratifiedKFold(n_splits=cv, random_state=seed)
    for train, valid in skf.split(x, y):
        clf.fit(x.iloc[train,:], y[train])
        train_pred, valid_pred = clf.predict_proba(x.iloc[train,:]), clf.predict_proba(x.iloc[valid,:])
        train_pred, valid_pred = calc_proba(train_pred),             calc_proba(valid_pred)
        auc     = pd.DataFrame({"k" : [k],
                                "train_"+name : roc_auc_score(y[train], train_pred),
                                "valid_"+name : roc_auc_score(y[valid], valid_pred)})
        df_auc  = df_auc.append(auc, ignore_index=True)
        df_pred = pd.DataFrame({"idx" : valid, name : valid_pred, "y" : y[valid]})
        df_train_pred = df_train_pred.append(df_pred, ignore_index=True)
        k += 1
    # test
    clf.fit(x, y)
    all_pred, test_pred = clf.predict_proba(x), clf.predict_proba(df_test)
    all_pred, test_pred = calc_proba(all_pred), calc_proba(test_pred)
    auc    = pd.DataFrame({"k" : ["all"],
                           "train_"+name : roc_auc_score(y, all_pred)})
    df_auc = df_auc.append(auc, ignore_index=True)
    df_test_pred = pd.DataFrame({"idx" : df_test.index, name : test_pred})
    print("clf:%s time:%s end" % (name, datetime.now() - statime))
    return df_train_pred, df_test_pred, df_auc

In [ ]:
# クロスバリデーション k=4 でスタッキング（ハイパーパラメータはGridSearchで決定）
seed=15
# 勾配ブースティングでモデル作成
classifier = xgb.XGBClassifier(learning_rate=0.01, max_depth=6, min_child_weight=4, gamma=3, n_estimators=1000, random_state=seed)
df_train_xgb, df_test_xgb, df_auc_xgb = stacking(df_train_add_var, df_test_add_var, classifier, "xgb")
# アダブーストでモデル作成
classifier = AdaBoostClassifier(learning_rate=0.1, n_estimators=3000, random_state=seed)
df_train_ab,  df_test_ab,  df_auc_ab  = stacking(df_train_add_var, df_test_add_var, classifier, "ab")
# ランダムフォレストでモデル作成
classifier = RandomForestClassifier(max_depth=24, min_samples_leaf=4, min_samples_split=5, n_estimators=1000, n_jobs=-1, random_state=seed)
df_train_rf,  df_test_rf,  df_auc_rf  = stacking(df_train_add_var, df_test_add_var, classifier, "rf")
# ExtraTreesでモデル作成
classifier = ExtraTreesClassifier(max_depth=24, min_samples_leaf=1, n_estimators=500, n_jobs=-1, random_state=seed)
df_train_et,  df_test_et,  df_auc_et  = stacking(df_train_add_var, df_test_add_var, classifier, "et")

In [ ]:
classifier = ExtraTreesClassifier(max_depth=24, min_samples_leaf=1, n_estimators=500, n_jobs=-1, random_state=seed)
df_train_et,  df_test_et,  df_auc_et  = stacking(df_train_add_var, df_test_add_var, classifier, "et")

In [ ]:
# 各モデルのAUC
#  xgb : 勾配ブースティング
#  ab  : アダブースト
#  rf  : ランダムフォレスト
#  et  : Extra-trees
pd.concat([df_auc_xgb, df_auc_ab, df_auc_rf, df_auc_et], axis=1)

In [ ]:
# train
df_train_sta = pd.merge(df_train_rf,  df_train_xgb, on=["idx","y"])
df_train_sta = pd.merge(df_train_sta, df_train_et,  on=["idx","y"])
df_train_sta = pd.merge(df_train_sta, df_train_ab,  on=["idx","y"])
df_train_sta = df_train_sta[["idx", "xgb", "ab", "rf", "et", "y"]]
# test
df_test_sta  = pd.merge(df_test_rf,  df_test_xgb, on="idx")
df_test_sta  = pd.merge(df_test_sta, df_test_et,  on="idx")
df_test_sta  = pd.merge(df_test_sta, df_test_ab,  on="idx")
df_test_sta  = df_test_sta[["idx", "xgb", "ab", "rf", "et"]]
print("train shape :", df_train_sta.shape)
print("test  shape :", df_test_sta.shape)
df_train_sta.head()

In [ ]:
x    = df_train_sta.drop(["idx","y"], axis=1)
y    = df_train_sta.y
test = df_test_sta.drop("idx", axis=1)
# meta model（ハイパーパラメータはGridSearchで決定）
classifier = xgb.XGBClassifier(gamma=10, learning_rate=0.1, max_depth=6, min_child_weight=10, n_estimators=500, random_state=seed)
classifier.fit(x, y)
train_pred = calc_proba(classifier.predict_proba(x))
print("train auc:", roc_auc_score(y, train_pred))
test_pred  = calc_proba(classifier.predict_proba(test))

In [ ]:
df_result = pd.DataFrame({"idx"    : np.arange(1, df_test_sta.shape[0]+1),
                          "result" : test_pred})
path = "../../../../study/bank/submit/"
df_result.to_csv(path + "result_2018070.csv", header=None, index=False)

In [ ]:
##########################################
# cross validation
seed  = 15

In [ ]:
#勾配ブースティング
classifier = xgb.XGBClassifier(random_state=seed)
parameters = {'n_estimators'     : [500],
              'learning_rate'    : [0.1,0.3,0.5], 
              'max_depth'        : [4,6,8],
              'min_child_weight' : [4,6,10],
              'gamma'            : [0,3,10]}

In [ ]:
#extra trees
classifier = ExtraTreesClassifier(random_state=seed)
parameters = {'n_estimators'     : [500, 1000],
              'max_depth'        : [24],
              'min_samples_leaf' : [1, 3, 5]}

In [ ]:
#ランダムフォレスト
classifier = RandomForestClassifier(random_state=seed)
parameters = {'n_estimators'     : [1000],
              'max_depth'        : [24],
              'min_samples_leaf' : [2,5,6,10],
              'min_samples_split': [2,10,15,20]}

In [ ]:
#k最近傍法
classifier = KNeighborsClassifier()
parameters = {'n_neighbors':[62,63,64,65,66,67,68,69,70],
              "leaf_size"  :[1],
              "p"          :[1]}

In [ ]:
#SVM
classifier = SVC(random_state=seed)
parameters = {'C'    :np.logspace(-4, 4, 3),
              'gamma':np.logspace(-4, 4, 3)}

In [ ]:
#アダブースト
classifier = AdaBoostClassifier(random_state=seed)
parameters = {'n_estimators'    :[3000,5000],
              'learning_rate'   :[0.1,0.2,0.3]}

In [ ]:
#ロジスティック回帰
classifier = LogisticRegression(random_state=seed)
parameters = {'C' : [0.1,1,10,20,30,40,50,60,70,80,90,100]}

In [ ]:
x = df_train_add_var.drop("y", axis=1)
y = df_train_add_var.y
train_X, test_X, train_y, test_y = train_test_split(x, y, 
                                                    test_size=0.2,
                                                    random_state=seed)

cv = GridSearchCV(classifier, parameters, cv=4, scoring="roc_auc", verbose=10, n_jobs=-1)
cv.fit(train_X, train_y)

In [ ]:
x = df_train_sta.drop(["idx","y"], axis=1)
y = df_train_sta.y
train_X, test_X, train_y, test_y = train_test_split(x, y, 
                                                    test_size=0.2,
                                                    random_state=seed)

cv = GridSearchCV(classifier, parameters, cv=4, scoring="roc_auc", verbose=10, n_jobs=-1)
cv.fit(train_X, train_y)

In [ ]:
pred = calc_proba(cv.predict_proba(test_X))
print("best model auc:", roc_auc_score(test_y, pred))
cv.best_estimator_.get_params

In [ ]:
############################################
#DeepLearning（適当）

In [ ]:
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.optimizers import RMSprop
import keras

In [ ]:
y_cate = keras.utils.np_utils.to_categorical(y,  2)

In [ ]:
model = keras.Sequential()
model.add(Dense(512, activation='relu', input_dim=x.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [ ]:
#学習の実行
epochs=128
batch_size=100
history1 = model.fit(x, y_cate,  epochs=epochs, batch_size=batch_size)

In [ ]:
# 年齢を年代に変換
#classes = [15,25,30,35,40,45,50,55,60,120]
#df_add_var["ageclass"] = pd.cut(df_add_var.age, classes, labels=np.arange(9)).astype(int)
# 年齢を50歳で折り返して線形にする
#df_add_var["age_abs"] = np.abs(50 - df_add_var.age)
# 経過日数を２か月、６か月、１年に分割する
#classes = [-2,0,60,120,180,360,720,2000]
#df_add_var["pdaysclass"] = pd.cut(df_add_var.pdays, classes, labels=np.arange(7)).astype(int)
# 新規顧客フラグ
#df_add_var["new_customers"] = 0
#df_add_var.loc[df_add_var.previous==0, "new_customers"] = 1
